# Project Template - adapted

## Imports

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

Be sure to start the stream on Kafka!

## Create Spark Session

In [2]:
spark = (
    SparkSession.builder
        .appName("Project DPA")
        .master("local[*]")   
        .config("spark.jars.packages",
                "org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0")
        .config("spark.sql.repl.eagerEval.enabled", True)
        .getOrCreate()
)

spark.sparkContext.setLogLevel("WARN")

Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
:: loading settings :: url = jar:file:/usr/local/lib/python3.9/dist-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4709af06-bc19-4f28-8987-3a7037117e55;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.0.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.0.0 in central
	found org.apache.kafka#kafka-clients;2.4.1 in central
	found com.github.luben#zstd-jni;1.4.4-3 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.7.5 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 172ms :: artifacts dl 6m

## Schema Definiton

In [3]:
schema = StructType([
    StructField("name", StringType()),
    StructField("price", FloatType()),
    StructField("n_shares", IntegerType()),
    StructField("timestamp", StringType())
])

# Kafka Stream Ingestion

In [4]:
# Read from Kafka topic "stock"
raw_df = (
    spark.readStream
         .format("kafka")
         .option("kafka.bootstrap.servers", "kafka1:9092")
         .option("subscribe", "stock")
         .option("startingOffsets", "latest")
         .load()
)

# Kafka sends bytes → convert to string
json_df = raw_df.selectExpr("CAST(value AS STRING)")

# Parse JSON using the schema
parsed_df = json_df.select(
    from_json(col("value"), schema).alias("data")
).select("data.*")

# Convert timestamp string into actual timestamp type
parsed_df = parsed_df.withColumn(
    "timestamp", to_timestamp("timestamp", "yyyy-MM-dd'T'HH:mm:ss'Z'")
)

# Make the dataset available to all following tasks
parsed_df.createOrReplaceTempView("stocks_stream")

# Start Streaming Output

This is optional but just to make sure that Spark is actually receiving data

In [ ]:
query = (
    parsed_df.writeStream
             .format("console")
             .outputMode("append")
             .option("truncate", False)
             .start()
)

## The assignment starts here

Remark: We consider a day being 5 minutes so we do not have to wait an actual day for the code results. Hence, an hour corresponds to 12.5 seconds.

### Task 5

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import time

TARGET_STOCK = "AAPL"   
MAX_WINDOWS = 20        
TAKE_PROFIT = 0.20
STOP_LOSS   = 0.05

filtered_df = parsed_df.filter(col("name") == TARGET_STOCK)

task5 = (
    filtered_df
        .withWatermark("timestamp", "5 minutes")
        .groupBy(
            window("timestamp", "5 minutes"),
            "name"
        )
        .agg(
            first("price").alias("entry_price"),
            last("price").alias("current_price"),
            count("*").alias("n_events")
        )
        .withColumn("gain_percentage",
            (col("current_price") - col("entry_price")) / col("entry_price")
        )
        .withColumn("take_profit_hit", col("gain_percentage") >= TAKE_PROFIT)
        .withColumn("stop_loss_hit",   col("gain_percentage") <= -STOP_LOSS)
)

task5_query = (
    task5.writeStream
         .format("console")
         .outputMode("update")
         .option("truncate", False)
         .start()
)

print(f"\n>>> STREAMING STARTED ONLY FOR {TARGET_STOCK}\n")

#automatic stop after N windows:
windows_seen = 0

while task5_query.isActive:

    time.sleep(2)

    try:
        
        current_count = (
            task5.select(count("*").alias("c")).collect()[0]["c"]
        )

        if current_count > windows_seen:
            windows_seen = current_count

        if windows_seen >= MAX_WINDOWS:
            print(f"\n>>> AUTOMATIC STOP: reached {MAX_WINDOWS} analyzed windows")
            task5_query.stop()
            break

    except:
        pass


25/11/25 11:51:39 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-ece997d9-118e-48c6-9639-ef957299fb3d. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.



>>> STREAMING AVVIATO SOLO PER AAPL



-------------------------------------------
Batch: 0
-------------------------------------------
+------+----+-----------+-------------+--------+---------------+---------------+-------------+
|window|name|entry_price|current_price|n_events|gain_percentage|take_profit_hit|stop_loss_hit|
+------+----+-----------+-------------+--------+---------------+---------------+-------------+
+------+----+-----------+-------------+--------+---------------+---------------+-------------+



-------------------------------------------
Batch: 1
-------------------------------------------
+------+----+-----------+-------------+--------+---------------+---------------+-------------+
|window|name|entry_price|current_price|n_events|gain_percentage|take_profit_hit|stop_loss_hit|
+------+----+-----------+-------------+--------+---------------+---------------+-------------+
+------+----+-----------+-------------+--------+---------------+---------------+-------------+



-------------------------------------------
Batch: 2
-------------------------------------------
+------+----+-----------+-------------+--------+---------------+---------------+-------------+
|window|name|entry_price|current_price|n_events|gain_percentage|take_profit_hit|stop_loss_hit|
+------+----+-----------+-------------+--------+---------------+---------------+-------------+
+------+----+-----------+-------------+--------+---------------+---------------+-------------+



...

-------------------------------------------
Batch: 400
-------------------------------------------
+------+----+-----------+-------------+--------+---------------+---------------+-------------+
|window|name|entry_price|current_price|n_events|gain_percentage|take_profit_hit|stop_loss_hit|
+------+----+-----------+-------------+--------+---------------+---------------+-------------+
+------+----+-----------+-------------+--------+---------------+---------------+-------------+



-------------------------------------------
Batch: 401
-------------------------------------------
+------+----+-----------+-------------+--------+---------------+---------------+-------------+
|window|name|entry_price|current_price|n_events|gain_percentage|take_profit_hit|stop_loss_hit|
+------+----+-----------+-------------+--------+---------------+---------------+-------------+
+------+----+-----------+-------------+--------+---------------+---------------+-------------+



-------------------------------------------
Batch: 402
-------------------------------------------
+------+----+-----------+-------------+--------+---------------+---------------+-------------+
|window|name|entry_price|current_price|n_events|gain_percentage|take_profit_hit|stop_loss_hit|
+------+----+-----------+-------------+--------+---------------+---------------+-------------+
+------+----+-----------+-------------+--------+---------------+---------------+-------------+



-------------------------------------------
Batch: 403
-------------------------------------------
+------+----+-----------+-------------+--------+---------------+---------------+-------------+
|window|name|entry_price|current_price|n_events|gain_percentage|take_profit_hit|stop_loss_hit|
+------+----+-----------+-------------+--------+---------------+---------------+-------------+
+------+----+-----------+-------------+--------+---------------+---------------+-------------+



-------------------------------------------
Batch: 404
-------------------------------------------
+------+----+-----------+-------------+--------+---------------+---------------+-------------+
|window|name|entry_price|current_price|n_events|gain_percentage|take_profit_hit|stop_loss_hit|
+------+----+-----------+-------------+--------+---------------+---------------+-------------+
+------+----+-----------+-------------+--------+---------------+---------------+-------------+

